<a href="https://colab.research.google.com/github/Bitang-Melyen-Tanulok/Csip_Csip/blob/main/Filter_spectrograms_with_Google_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below uses the Google Bird Vocalization Classifier to filter the likely incorrect data.
If the prediction of the model is far from the correct bird species (even when it comes to the family of the species), we get rid of it.

If any secondary label comes out as the prediction of the model, we switch out the primary label for the secondary one.

In [3]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/DeepLearning'

# Labels file path
labels_path = '/content/drive/MyDrive/DeepLearning/label.csv'
labels_df = pd.read_csv(labels_path, header=None)
labels = labels_df[0].tolist()  # The first column contains the labels
print(f"Number of labels read: {len(labels)}")

# Load the bird vocalization classifier model
model = hub.load('https://tfhub.dev/google/bird-vocalization-classifier/1')

# Define paths for data
audio_path = os.path.join(base_path, 'sample_train_audio')  # Directory for audio files
spectrograms_path = os.path.join(base_path, 'ugyanaz?')  # Directory for spectrogram files
metadata_path = os.path.join(base_path, 'train_metadata.csv')  # Path to metadata file

# Load metadata
metadata = pd.read_csv(metadata_path)

print(labels[:10])  # Print the first 10 labels
print(metadata['primary_label'].unique()[:10])  # Print the first 10 unique primary labels

# Function to log actions and handle file deletion/renaming
def log_and_act(file, file_path, ogg_file_path, predicted_label_index, primary_label, secondary_labels, decision, final_label=None):
    #print(f"File: {file}")
    #print(f"   - Primary label: {primary_label}")
    #print(f"   - Secondary labels: {secondary_labels}")
    #print(f"   - Predicted label index: {predicted_label_index}")
    #print(f"   - Decision: {decision}")
    #if final_label:
    #    print(f"   - Final label: {final_label}")

    # Handle deletion or renaming based on the decision
    if decision.startswith("Skipped"):
        # Delete the spectrogram
        os.remove(file_path)  # Delete spectrogram file
        print(f"   - Deleted: {file_path}")
    elif decision.startswith("Kept (label updated)"):
        # Rename spectrogram with updated label
        new_folder = os.path.join(spectrograms_path, final_label)
        os.makedirs(new_folder, exist_ok=True)
        new_file_path = os.path.join(new_folder, os.path.basename(file_path))
        os.rename(file_path, new_file_path)
        print(f"   - Renamed: {file_path} -> {new_file_path}")
    #print()

# Process audio files and filter them
folders = os.listdir(spectrograms_path)
for folder in folders:
    print(f"Current folder: {folder}")
    folder_path = os.path.join(spectrograms_path, folder)

    # Load spectrogram files
    files = os.listdir(folder_path)
    for file in files:
        if file.endswith('.png'):
            file_path = os.path.join(folder_path, file)
            ogg_file_path = os.path.join(audio_path, f"{folder}/{file.replace('.png', '.ogg')}")

            # Skip files if the corresponding audio file does not exist
            if not os.path.exists(ogg_file_path):
                log_and_act(file, file_path, ogg_file_path, None, None, None, "Skipped (ogg file not found)")
                continue

            # Get metadata for the file
            row = metadata[metadata['filename'] == f"{folder}/{file.replace('.png', '.ogg')}"]
            if row.empty:
                log_and_act(file, file_path, ogg_file_path, None, None, None, "Skipped (metadata not found)")
                continue

            primary_label = row['primary_label'].values[0]
            secondary_labels = eval(row['secondary_labels'].values[0])  # Convert to list

            # Load the audio file
            waveform, _ = librosa.load(ogg_file_path, sr=32000)

            # Pad with zeros if the file is less than 5 seconds
            if len(waveform) < 5 * 32000:
                waveform = np.pad(waveform, (0, 5 * 32000 - len(waveform)), mode='constant')
            waveform = waveform[:5 * 32000]  # Truncate to 5 seconds if longer

            # Run inference with the model
            predictions = model.infer_tf(waveform[np.newaxis, :])[0].numpy()
            predicted_label_index = np.argmax(predictions)
            predicted_label = labels[predicted_label_index]

            #print(f"Predicted label index: {predicted_label_index}")
            #print(f"Predicted label: {predicted_label}")

            # Filtering logic: keep or skip based on predictions
            if predicted_label[0] == primary_label[0] or predicted_label in secondary_labels:
                decision = "Kept"
                final_label = primary_label
                if predicted_label in secondary_labels:
                    decision = "Kept (label updated)"
                    final_label = predicted_label

                log_and_act(file, file_path, ogg_file_path, predicted_label_index, primary_label, secondary_labels, decision, final_label)
            else:
                log_and_act(file, file_path, ogg_file_path, predicted_label_index, primary_label, secondary_labels, "Skipped (label mismatch)")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of labels read: 10933
['ebird2021', 'aakspa1', 'abbbab1', 'abbboo2', 'abbsta2', 'abbwar1', 'abcwin1', 'abdsto1', 'abecis1', 'abethr1']
['asbfly' 'ashdro1' 'ashpri1' 'ashwoo2' 'asikoe2' 'asiope1' 'aspfly1'
 'aspswi1' 'barfly1' 'barswa']
Current folder: blrwar1
Current folder: grenig1
Current folder: junmyn1
Current folder: pursun4
Current folder: revbul
Current folder: rutfly6
Current folder: barswa
   - Deleted: /content/drive/MyDrive/DeepLearning/ugyanaz?/barswa/XC770651.png
   - Deleted: /content/drive/MyDrive/DeepLearning/ugyanaz?/barswa/XC472519.png
   - Deleted: /content/drive/MyDrive/DeepLearning/ugyanaz?/barswa/XC592362.png
   - Deleted: /content/drive/MyDrive/DeepLearning/ugyanaz?/barswa/XC632301.png
   - Deleted: /content/drive/MyDrive/DeepLearning/ugyanaz?/barswa/XC669886.png
   - Deleted: /content/drive/MyDrive/DeepLearning/ugyanaz?/barswa/X

rutfly6 gets completely deleted, probably because of bad data (the classifier is able to recognize this class)